Our dataset is composed of multiple different timeseries, which may not share the same length. As per the [documentation](https://github.com/reservoirpy/reservoirpy/blob/master/tutorials/1-Getting_Started.ipynb)